In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso, MultiTaskLassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (GridSearchCV,cross_val_score, KFold)
from sklearn.pipeline import Pipeline


train_features = pd.read_csv('/content/drive/MyDrive/ml_task2/train_features.csv') #X
train_labels = pd.read_csv('/content/drive/MyDrive/ml_task2/train_labels.csv') #y
test_features = pd.read_csv('/content/drive/MyDrive/ml_task2/test_features.csv')

#train_y = train_features['y']
#train_X = data.drop('y',axis=1)

y = train_labels.sort_values(by=['pid'])

y.reset_index(drop=True, inplace=True)
y_train = y.iloc[:,0:11].drop('pid', axis=1)
X = train_features
#['BaseExcess', 'Fibrinogen','AST', 'Alkalinephos', 'Bilirubin_total', 'Lactate', 'TroponinI', 'SaO2', 'Bilirubin_direct', 'EtCO2']

### **Feauture selection**

In [5]:
X=X.interpolate(method ='linear',limit_direction='both')
X_mean = X.groupby(['pid']).mean()
X_last=X.groupby(['pid']).last() 
u= y_train.to_numpy(copy=True)

X_test = test_features.interpolate(method ='linear',limit_direction='both')
X_test_mean = X_test.groupby(['pid']).mean()
X_test_last=X_test.groupby(['pid']).last() 

### **Subtask 1**
Train Pipeline (This doesn't give probabilsitc values, try different method)

In [6]:
y_pred = np.zeros([np.size(X_test_mean,0),10])

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
for i in range(10):
  X_train, X_test, y_train, y_test = train_test_split(X_mean, u[:,i], random_state=0)
  pipe.fit(X_train, y_train)
  y_pred[:,i]=pipe.predict(X_test_mean)

Save subtask 1


In [24]:
tests = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
df1 = pd.DataFrame(y_pred, index=X_test_mean.index, columns=tests)

In [27]:
df1.to_csv('subtask_1.csv')

### **Subtask 2**
Predict Sepsis.

**DID**: no feature section

**TODO**: Check for (min two): abnormal body temperature, heart rate, respiratory rate, or blood gas, and white blood cell count.


In [7]:
sepsis = y['LABEL_Sepsis']

pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe_s = pipe.fit(X_mean, sepsis)
sepsis_pred = pipe.predict(X_test_mean)


In [28]:
df2 = pd.DataFrame(sepsis_pred, index=X_test_mean.index, columns=['LABEL_Sepsis'])
df2.to_csv('subtask_2.csv')

### **Subtask 3**
Regression. LABEL_RRate, LABEL_ABPm, LABEL_SpO2, LABEL_Heartrate

**DID:**

**TODO:**

In [ ]:
y3 = y[['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']].to_numpy(copy=True)

model = RidgeCV().fit(X_last,y3)
y3_pred = model.predict(X_test_last)

### **SAVE**

In [ ]:
y_pred = pd.DataFrame(y_pred)
sepsis_pred = pd.DataFrame(sepsis_pred)
y3_pred = pd.DataFrame(y3_pred)
V= X_test_last.copy()
index = pd.DataFrame(V.index)
submission=pd.concat([index,y_pred,sepsis_pred,y3_pred],axis=1).set_axis(['pid','LABEL_BaseExcess'	,'LABEL_Fibrinogen',	'LABEL_AST',	'LABEL_Alkalinephos',	'LABEL_Bilirubin_total'	,'LABEL_Lactate',	'LABEL_TroponinI',	'LABEL_SaO2',	'LABEL_Bilirubin_direct',	'LABEL_EtCO2',	'LABEL_Sepsis',	'LABEL_RRate',	'LABEL_ABPm',	'LABEL_SpO2',	'LABEL_Heartrate'], axis=1)
#df.to_csv('submission.csv')
submission.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')
submission


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,17.135720,83.086191,97.010189,98.513582
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.307031,86.528473,98.117169,89.835671
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.553944,74.426114,95.986770,76.499491
3,7,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,18.114315,86.143543,97.380449,90.935827
4,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.197397,93.688496,96.771115,87.666633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12659,31647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.191869,81.110476,97.351311,76.154387
12660,31649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.473438,90.031544,96.366755,86.771283
12661,31651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.911917,79.856808,97.772779,92.641397
12662,31652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.109995,87.296809,96.829522,102.738420


## **TEST SECTION**
DONT LOOK pls

In [ ]:
sample = pd.read_csv('/content/sample.csv')
sample
test_features.groupby(['pid']).last()

AttributeError: ignored

In [ ]:
import math
X_mod=X.copy(deep=True)
for i in range(7): #227940/12.   18995
  for k in range(np.size(X,axis=1)):
    for j in range(12):
      if math.isnan(X_mod.iloc[12*i+j,k]) and j==0:
        m=1
        while m<11 and math.isnan(X_mod.iloc[12*i+j+m,k]):
          m = m+1
        X_mod.iat[12*i+j,k]=X_mod.iloc[12*i+j+m,k]
      if math.isnan(X_mod.iloc[12*i+j,k]) and j!=0:
        X_mod.iat[12*i+j,k]=X_mod.iloc[12*i+j-1,k]
X_mod.head(7*12)